In [ ]:
# Installs requirements in case they are missing
%pip install spacy gensim pyLDAvis
import sys
import os

if sys.platform.startswith('win'):
    os.system('python -m spacy download en_core_web_md')
elif sys.platform.startswith('darwin') or sys.platform.startswith('linux'):
    os.system('python3 -m spacy download en_core_web_md')

In [12]:
import glob
import spacy
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim
from IPython.display import display
import pyLDAvis

pyLDAvis.enable_notebook()

nlp = spacy.load('en_core_web_sm')

In [13]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if not token.is_stop and not token.is_punct and token.lemma_.isalpha():
            filtered_tokens.append(token.lemma_)
    return filtered_tokens

def create_lda_model(file_paths):
    documents = []
    for path_book_name in file_paths:
        with open(path_book_name, 'r') as file:
            documents.append(file.read())

    cleaned_texts = []
    for doc in documents:
        cleaned_texts.append(preprocess(doc))

    dictionary = corpora.Dictionary(cleaned_texts)
    corpus = [dictionary.doc2bow(text) for text in cleaned_texts]

    lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)
    
    return lda_model, corpus, dictionary


The Intertopic Distance Map is a visualization tool that is often used in topic modeling to show the relationships between different topics.
In the graphs below each topic is represented as a bubble, and the distance between the bubbles represents the similarity between the topics. Topics that are closer together are more similar. The size of the bubble represents the prevalence of the topic in the corpus.
Information Source: pyLDAvis documentation

In [14]:
# Create LDA model for Marlowe books
marlowe_files = glob.glob("books/Marlowe-corpus/*.txt")
marlowe_lda_model, marlowe_corpus, marlowe_dictionary = create_lda_model(marlowe_files)
marlowe_vis = pyLDAvis.gensim.prepare(marlowe_lda_model, marlowe_corpus, marlowe_dictionary)

# Create LDA model for Shakespeare books
shakespeare_files = glob.glob("books/Shakespeare-corpus/*.txt")
shakespeare_lda_model, shakespeare_corpus, shakespeare_dictionary = create_lda_model(shakespeare_files)
shakespeare_vis = pyLDAvis.gensim.prepare(shakespeare_lda_model, shakespeare_corpus, shakespeare_dictionary)

# Display the visualizations
print("Marlowe Corpus Topics:")
pyLDAvis.display(marlowe_vis)

print("Shakespeare Corpus Topics:")
pyLDAvis.display(shakespeare_vis)

Marlowe Corpus Topics:
Shakespeare Corpus Topics:


In [31]:
if len(glob.glob("books/custom-corpus/*.txt")) > 0:
    custom_lda_model, custom_corpus, custom_dictionary = create_lda_model(glob.glob("books/custom-corpus/*.txt"))

    # Display the visualization
    print("Custom Corpus Topics:")
    display(pyLDAvis.gensim.prepare(custom_lda_model, custom_corpus, custom_dictionary))
else:
    print("No text files found in the custom-corpus directory.")

Custom Corpus Topics:


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
3     -0.116005 -0.0       1        1  99.990692
0      0.012992 -0.0       2        1   0.001034
1      0.012580 -0.0       3        1   0.001034
2      0.013442  0.0       4        1   0.001034
4      0.013098  0.0       5        1   0.001034
5      0.013549  0.0       6        1   0.001034
6      0.013621 -0.0       7        1   0.001034
7      0.012174 -0.0       8        1   0.001034
8      0.013151 -0.0       9        1   0.001034
9      0.011397  0.0      10        1   0.001034, topic_info=        Term        Freq       Total Category  logprob  loglift
2970    thou  149.000000  149.000000  Default  30.0000  30.0000
2984     thy  152.000000  152.000000  Default  29.0000  29.0000
1893    lord   92.000000   92.000000  Default  28.0000  28.0000
2952    thee   89.000000   89.000000  Default  27.0000  27.0000
2648   shall   81.000000   81.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
476   alarum    0.000032   17.500887  Topic10  -8.0475  -1.7325
91      Duke    0.000032   25.239998  Topic10  -8.0403  -2.0915
1663  honour    0.000032   19.435146  Topic10  -8.0458  -1.8356
2099       o    0.000032   21.369004  Topic10  -8.0439  -1.9286
1391     fly    0.000032   26.204663  Topic10  -8.0445  -2.1332

[681 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
21        1  0.941438       BASSET
35        1  0.941452  Bolingbroke
38        1  0.941243     Brandish
47        1  0.941284         Cain
55        1  0.986965      Charles
...     ...       ...          ...
3258      1  0.941382   winchester
3259      1  0.941446         wind
3274      1  0.941427     withhold
3302      1  0.941401   worthiness
3319      1  0.984863           ye

[358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5, 6, 7, 8, 9, 10])